<a href="https://colab.research.google.com/github/jmoriwa/ai_agents_workspace/blob/main/document_drafter_ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langgraph
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75


In [9]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage #Foundational class for all message types in LangGraph
from langchain_core.messages import ToolMessage, HumanMessage # Passes data back to LLM after it calls a tool such as content and the tool_call_id
from langchain_core.messages import SystemMessage # Message for providing instructions to the LLM
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from google.colab import userdata
import os
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key


In [10]:
#This is a global variable to store document content
document_content = ""

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]


@tool
def update(content: str) -> str:
  """Updates the document with provided content"""
  global document_content
  document_content = content
  return f"Document has been updates successfully! The current content is:\n{document_content}"

@tool
def save(filename:str) -> str:
  """Save the current document to a text file and finish the process

  Args:
      filename: Name for the text file.
  """
  if not filename.endswith('.txt'):
    filename = f"{filename}.txt"

  try:
    with open(filename, 'w') as file:
      file.write(document_content)
    print(f"\n💾 document has been saved to '{filename}'.")
    return f"Document has been saved successfull to '{filename}'"

  except Exception as e:
    return f"An error occurred while saving the document: {str(e)}"

tools = [update,save]

model = ChatOpenAI(model="gpt-4o").bind_tools(tools)


In [24]:
def our_agent(state: AgentState) -> AgentState:
  system_prompt = SystemMessage(content=f"""
  You are a drafter, a helpful writing assistant. You are going to help the user update and modify documents.

  - If the user wants to update or modify content, se the 'update' tool with the complete updated content.
  - If the user wants to save and finish, you need to use the 'save' tool
  - Make sure to always show the current document state after modification

  The current document content is:{document_content}
  """)

  if not state["messages"]:
    user_input = "I'm ready to help you update a document. Would you like to create?"
    user_message = HumanMessage(content= user_input)

  else:
    user_input = input("\nWhat would you like to do with the document? ")
    print(f"\n  USER: {user_input}")
    user_message = HumanMessage(content=user_input)

  all_messages = [system_prompt] + list(state["messages"]) + [user_message]

  response = model.invoke(all_messages)

  print(f"\n🤖 AI: {response.content}")
  if hasattr(response, "tool_calls") and response.tool_calls:
    print(f"🔨 USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")

  return {"messages": list(state["messages"]) + [user_message, response]}

In [25]:
def should_continue(state: AgentState) -> str:
  """ Determine if we should continue or end the conversation"""
  messages = state["messages"]

  if not messages:
    return "continue"

  # This looks for the most recent tool call
  for message in reversed(messages):
    # ... and checks if this is a ToolMessage resulting from save
    if (isinstance(message, ToolMessage) and
        "saved" in message.content.lower() and "document" in message.content.lower()):
      return "end"

  return "continue"


def print_messages(messages):
  """Function to print messsages in a more readable format"""
  if not messages:
    return

  for message in messages[-3:]:
    if isinstance(message, ToolMessage):
      print(f"\n🛠️ TOOL RESULT: {message.content}")


In [26]:
graph = StateGraph(AgentState)

graph.add_node("agent", our_agent)
graph.add_node("tools", ToolNode(tools))

graph.set_entry_point("agent")

graph.add_edge("agent", "tools")

graph.add_conditional_edges(
    "tools",
    should_continue,{
        "continue": "agent",
        "end": END
    },
)

app = graph.compile()

In [27]:
def run_document_agent():
  print("\n ==== DRAFTER ====")

  state = {"messages": []}

  for step in app.stream(state, stream_mode="values"):
    if "messages" in step:
      print_messages(step["messages"])

  print("\n ==== DRAFTER FINISHED ====")


run_document_agent()


 ==== DRAFTER ====

🤖 AI: Sure, I can help with creating or updating a document. Please provide the content you wish to add or modify in the document.

What would you like to do with the document? Write me an email to Jack saying I cannot make it to the meeting

  USER: Write me an email to Jack saying I cannot make it to the meeting

🤖 AI: 
🔨 USING TOOLS: ['update']

🛠️ TOOL RESULT: Document has been updates successfully! The current content is:
Subject: Unable to Attend Meeting

Hi Jack,

I hope this message finds you well. I am writing to inform you that I will not be able to attend the meeting we had scheduled. Please accept my apologies for any inconvenience this may cause.

Thank you for your understanding and support. Let me know if there are any updates or if I need to follow up on any actions from the meeting.

Best regards,

[Your Name]

What would you like to do with the document? make sure to also have specified the meeting was supposed to be at 10:00 am at the four season